In [2]:
from __future__ import absolute_import 
from __future__ import division
from __future__ import print_function

import numpy as np 
import os 
from imutils import paths
import cv2



from tensorflow.keras import backend
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
from tensorflow.keras import utils
from tensorflow.keras import optimizers

from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


L2_WEIGHT_DECAY    = 1e-4
BATCH_NORM_DECAY    = 0.9
BATCH_NORM_EPSILON = 1e-5

HEIGHT       = 28
WIDTH        = 28
CHANNEL    = 1

OFFSET       = 1

BATCH_SIZE  = 32


path       = './data2/trainingSet'
path_model  = './mnistNet.hdf5'
LABEL    = np.array(['0','1','2','3','4','5','6','7','8','9'])


FIXED_BATCH_SIZE = True

def read_mnist_data(paths, label, one_hot = True, dtype = 'float32'):
    imgs    = np.empty((len(paths), HEIGHT,WIDTH, CHANNEL), dtype=dtype)
    if one_hot:
        tgs    = np.empty((len(paths), len(label)))   
    else:
        tgs    = np.empty((len(paths), 1))

    for (i, path) in enumerate(paths):
        print("[INFO] processing image {}/{}".format(i + 1, len(paths)))
        name     = path.split(os.path.sep)#[-2]
        image    = cv2.imread(path)


    if CHANNEL == 1:
        im       = np.array(image[:,:,0], dtype = dtype)    # shape =  (28, 28)
        im       = np.expand_dims(im, axis = 2)             # shape =  (28, 28, 1)
    else:
        im       = np.array(image, dtype = dtype)    # shape =  (28, 28)

      
        idx = np.where(LABEL == name[-2])[0][0]
        if one_hot:
            tgs[i,:]    = 0.
            tgs[i,idx]    = 1.
        else:
    
            tgs[i] = idx
    

            imgs[i] = im
            #print(path)
            #print(name[-2], np.shape(im), np.shape(image))
            #print(tgs[i])

    index    = np.random.permutation(imgs.shape[0])
    imgs    = imgs[index]
    tgs    = tgs[index]

    if dtype == 'float32': imgs /= 255.
    return imgs, tgs



def read_mnist_data_onehot(path_train, path_truth):
    imagePaths       = list(paths.list_images(path_train))
    truethPaths    = list(paths.list_images(path_truth))

    images          = np.empty((len(imagePaths), HEIGHT, WIDTH, CHANNEL), dtype = 'float')
    truths          = np.empty((len(imagePaths), HEIGHT, WIDTH, 11), dtype = 'int')

    for (i, path) in enumerate(imagePaths):
        img = cv2.imread(path)

        im       = np.array(img[:,:,0], dtype =  'float')    # shape =  (28, 28)
        im       = np.expand_dims(im, axis = 2)          # shape =  (28, 28, 1)

        images[i]    = im / 255.

        path_split    = path.split('trainingSample')
        path       = path_split[0] + 'trainingSampleSeg' + path_split[1].split('.jpg')[0] + '.png'

        img       = cv2.imread(path)

        im          = np.array(img[:,:,0], dtype = 'int')    # shape =  (28, 28)

        truths[i]    = (np.arange(self.nlabels) == im[..., None]).astype(int)

    return images, truths   



def vgg(N_LABELS,  batch_size = None, dtype='float32'):
    input_shape = (HEIGHT, WIDTH, CHANNEL)
    convdepth    = [8, 16, 32, 64]
    fcdepth    = [256, 128]
    BNaxis       = 3   



    img_input = layers.Input(    shape = input_shape, 
                        dtype = dtype, 
                        batch_size = batch_size)   


    x = layers.Conv2D(   convdepth[0], (3, 3),
                  strides    = (1, 1),
                  padding    = 'same', 
                  use_bias    = True,
                  bias_initializer    = 'zeros',
                  kernel_initializer    = 'he_normal',
                  #kernel_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  name='conv1_1')(img_input) 


    x = layers.BatchNormalization(   axis     = BNaxis,
                           momentum = BATCH_NORM_DECAY,
                           epsilon  = BATCH_NORM_EPSILON,
                           name     = 'bn1_1')(x)      

    x = layers.Activation('relu')(x)            

    x = layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)


    x = layers.Conv2D(   convdepth[1], (3, 3),
                  strides    = (1, 1),
                  padding    = 'same', 
                  use_bias    = True,
                  bias_initializer    = 'zeros',
                  kernel_initializer    = 'he_normal',
                  #kernel_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  name='conv2_1')(x) 


    x = layers.BatchNormalization(   axis     = BNaxis,
                           momentum = BATCH_NORM_DECAY,
                           epsilon  = BATCH_NORM_EPSILON,
                           name     = 'bn2_1')(x)      

    x = layers.Activation('relu')(x)            

    x = layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)


    x = layers.Conv2D(   convdepth[2], (3, 3),
                  strides    = (1, 1),
                  padding    = 'same', 
                  use_bias    = True,
                  bias_initializer    = 'zeros',
                  kernel_initializer    = 'he_normal',
                  #kernel_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  name='conv3_1')(x) 


    x = layers.BatchNormalization(   axis     = BNaxis,
                           momentum = BATCH_NORM_DECAY,
                           epsilon  = BATCH_NORM_EPSILON,
                           name     = 'bn3_1')(x)      

    x = layers.Activation('relu')(x)            

    x = layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)


    x = layers.Conv2D(   convdepth[3], (3, 3),
                  strides    = (1, 1),
                  padding    = 'same', 
                  use_bias    = True,
                  bias_initializer    = 'zeros',
                  kernel_initializer    = 'he_normal',
                  #kernel_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  name='conv4_1')(x) 


    x = layers.BatchNormalization(   axis     = BNaxis,
                           momentum = BATCH_NORM_DECAY,
                           epsilon  = BATCH_NORM_EPSILON,
                           name     = 'bn4_1')(x)      

    x = layers.Activation('relu')(x)            

    x = layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)

    x = layers.Flatten()(x)

    x = layers.Dense(    fcdepth[0],  
                  kernel_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  bias_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  name='fc_1')(x)   

    x = layers.Dense(    fcdepth[1],  
                  kernel_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  bias_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  name='fc_2')(x)

    x = layers.Dense(    N_LABELS,  
                  kernel_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  bias_regularizer    = regularizers.l2(L2_WEIGHT_DECAY),
                  name='fc_3')(x)
    x = backend.cast(x, 'float32')

    x = layers.Activation('softmax')(x)

    return models.Model(img_input, x, name = 'vgg')






def train():
    imagePaths       = list(paths.list_images(path))
    #for name in imagePaths: print(name)
    X, Y = read_mnist_data(imagePaths, LABEL)

    Ntot    = len(X); 
    RATIO    = 0.66
    trX    = X[:int(Ntot*RATIO),...]
    teX    = X[int(Ntot*RATIO):,...]
    trY    = Y[:int(Ntot*RATIO),...]
    teY    = Y[int(Ntot*RATIO):,...]

    # this is for fixing batch size!!!
    trX = trX[:-(len(trX) % BATCH_SIZE),...]
    trY = trY[:-(len(trY) % BATCH_SIZE),...]
    teX = teX[:-(len(teX) % BATCH_SIZE),...]
    teY = teY[:-(len(teY) % BATCH_SIZE),...]


    datagen    = ImageDataGenerator()
    datagen.fit(trX)

    if FIXED_BATCH_SIZE:
        mymodel = vgg(len(LABEL), BATCH_SIZE)
    else:
        mymodel = vgg(len(LABEL), None)

    mymodel.compile(optimizer=optimizers.Adam(lr=0.001), loss='categorical_crossentropy',metrics=['accuracy'])
    mymodel.summary()


    mymodel.fit_generator(   datagen.flow(trX, trY, batch_size = BATCH_SIZE),
                     validation_data = (teX, teY), 
                      steps_per_epoch = int(len(trX) / BATCH_SIZE), epochs=10)

    mymodel.save(path_model)



if __name__ == '__main__':
    train()
   #inference()

[INFO] processing image 1/39299
[INFO] processing image 2/39299
[INFO] processing image 3/39299
[INFO] processing image 4/39299
[INFO] processing image 5/39299
[INFO] processing image 6/39299
[INFO] processing image 7/39299
[INFO] processing image 8/39299
[INFO] processing image 9/39299
[INFO] processing image 10/39299
[INFO] processing image 11/39299
[INFO] processing image 12/39299
[INFO] processing image 13/39299
[INFO] processing image 14/39299
[INFO] processing image 15/39299
[INFO] processing image 16/39299
[INFO] processing image 17/39299
[INFO] processing image 18/39299
[INFO] processing image 19/39299
[INFO] processing image 20/39299
[INFO] processing image 21/39299
[INFO] processing image 22/39299
[INFO] processing image 23/39299
[INFO] processing image 24/39299
[INFO] processing image 25/39299
[INFO] processing image 26/39299
[INFO] processing image 27/39299
[INFO] processing image 28/39299
[INFO] processing image 29/39299
[INFO] processing image 30/39299
[INFO] processing i

[INFO] processing image 291/39299
[INFO] processing image 292/39299
[INFO] processing image 293/39299
[INFO] processing image 294/39299
[INFO] processing image 295/39299
[INFO] processing image 296/39299
[INFO] processing image 297/39299
[INFO] processing image 298/39299
[INFO] processing image 299/39299
[INFO] processing image 300/39299
[INFO] processing image 301/39299
[INFO] processing image 302/39299
[INFO] processing image 303/39299
[INFO] processing image 304/39299
[INFO] processing image 305/39299
[INFO] processing image 306/39299
[INFO] processing image 307/39299
[INFO] processing image 308/39299
[INFO] processing image 309/39299
[INFO] processing image 310/39299
[INFO] processing image 311/39299
[INFO] processing image 312/39299
[INFO] processing image 313/39299
[INFO] processing image 314/39299
[INFO] processing image 315/39299
[INFO] processing image 316/39299
[INFO] processing image 317/39299
[INFO] processing image 318/39299
[INFO] processing image 319/39299
[INFO] process

[INFO] processing image 572/39299
[INFO] processing image 573/39299
[INFO] processing image 574/39299
[INFO] processing image 575/39299
[INFO] processing image 576/39299
[INFO] processing image 577/39299
[INFO] processing image 578/39299
[INFO] processing image 579/39299
[INFO] processing image 580/39299
[INFO] processing image 581/39299
[INFO] processing image 582/39299
[INFO] processing image 583/39299
[INFO] processing image 584/39299
[INFO] processing image 585/39299
[INFO] processing image 586/39299
[INFO] processing image 587/39299
[INFO] processing image 588/39299
[INFO] processing image 589/39299
[INFO] processing image 590/39299
[INFO] processing image 591/39299
[INFO] processing image 592/39299
[INFO] processing image 593/39299
[INFO] processing image 594/39299
[INFO] processing image 595/39299
[INFO] processing image 596/39299
[INFO] processing image 597/39299
[INFO] processing image 598/39299
[INFO] processing image 599/39299
[INFO] processing image 600/39299
[INFO] process

[INFO] processing image 866/39299
[INFO] processing image 867/39299
[INFO] processing image 868/39299
[INFO] processing image 869/39299
[INFO] processing image 870/39299
[INFO] processing image 871/39299
[INFO] processing image 872/39299
[INFO] processing image 873/39299
[INFO] processing image 874/39299
[INFO] processing image 875/39299
[INFO] processing image 876/39299
[INFO] processing image 877/39299
[INFO] processing image 878/39299
[INFO] processing image 879/39299
[INFO] processing image 880/39299
[INFO] processing image 881/39299
[INFO] processing image 882/39299
[INFO] processing image 883/39299
[INFO] processing image 884/39299
[INFO] processing image 885/39299
[INFO] processing image 886/39299
[INFO] processing image 887/39299
[INFO] processing image 888/39299
[INFO] processing image 889/39299
[INFO] processing image 890/39299
[INFO] processing image 891/39299
[INFO] processing image 892/39299
[INFO] processing image 893/39299
[INFO] processing image 894/39299
[INFO] process

[INFO] processing image 1142/39299
[INFO] processing image 1143/39299
[INFO] processing image 1144/39299
[INFO] processing image 1145/39299
[INFO] processing image 1146/39299
[INFO] processing image 1147/39299
[INFO] processing image 1148/39299
[INFO] processing image 1149/39299
[INFO] processing image 1150/39299
[INFO] processing image 1151/39299
[INFO] processing image 1152/39299
[INFO] processing image 1153/39299
[INFO] processing image 1154/39299
[INFO] processing image 1155/39299
[INFO] processing image 1156/39299
[INFO] processing image 1157/39299
[INFO] processing image 1158/39299
[INFO] processing image 1159/39299
[INFO] processing image 1160/39299
[INFO] processing image 1161/39299
[INFO] processing image 1162/39299
[INFO] processing image 1163/39299
[INFO] processing image 1164/39299
[INFO] processing image 1165/39299
[INFO] processing image 1166/39299
[INFO] processing image 1167/39299
[INFO] processing image 1168/39299
[INFO] processing image 1169/39299
[INFO] processing im

...

[INFO] processing image 38383/39299
[INFO] processing image 38384/39299
[INFO] processing image 38385/39299
[INFO] processing image 38386/39299
[INFO] processing image 38387/39299
[INFO] processing image 38388/39299
[INFO] processing image 38389/39299
[INFO] processing image 38390/39299
[INFO] processing image 38391/39299
[INFO] processing image 38392/39299
[INFO] processing image 38393/39299
[INFO] processing image 38394/39299
[INFO] processing image 38395/39299
[INFO] processing image 38396/39299
[INFO] processing image 38397/39299
[INFO] processing image 38398/39299
[INFO] processing image 38399/39299
[INFO] processing image 38400/39299
[INFO] processing image 38401/39299
[INFO] processing image 38402/39299
[INFO] processing image 38403/39299
[INFO] processing image 38404/39299
[INFO] processing image 38405/39299
[INFO] processing image 38406/39299
[INFO] processing image 38407/39299
[INFO] processing image 38408/39299
[INFO] processing image 38409/39299
[INFO] processing image 3841

[INFO] processing image 38672/39299
[INFO] processing image 38673/39299
[INFO] processing image 38674/39299
[INFO] processing image 38675/39299
[INFO] processing image 38676/39299
[INFO] processing image 38677/39299
[INFO] processing image 38678/39299
[INFO] processing image 38679/39299
[INFO] processing image 38680/39299
[INFO] processing image 38681/39299
[INFO] processing image 38682/39299
[INFO] processing image 38683/39299
[INFO] processing image 38684/39299
[INFO] processing image 38685/39299
[INFO] processing image 38686/39299
[INFO] processing image 38687/39299
[INFO] processing image 38688/39299
[INFO] processing image 38689/39299
[INFO] processing image 38690/39299
[INFO] processing image 38691/39299
[INFO] processing image 38692/39299
[INFO] processing image 38693/39299
[INFO] processing image 38694/39299
[INFO] processing image 38695/39299
[INFO] processing image 38696/39299
[INFO] processing image 38697/39299
[INFO] processing image 38698/39299
[INFO] processing image 3869

[INFO] processing image 38961/39299
[INFO] processing image 38962/39299
[INFO] processing image 38963/39299
[INFO] processing image 38964/39299
[INFO] processing image 38965/39299
[INFO] processing image 38966/39299
[INFO] processing image 38967/39299
[INFO] processing image 38968/39299
[INFO] processing image 38969/39299
[INFO] processing image 38970/39299
[INFO] processing image 38971/39299
[INFO] processing image 38972/39299
[INFO] processing image 38973/39299
[INFO] processing image 38974/39299
[INFO] processing image 38975/39299
[INFO] processing image 38976/39299
[INFO] processing image 38977/39299
[INFO] processing image 38978/39299
[INFO] processing image 38979/39299
[INFO] processing image 38980/39299
[INFO] processing image 38981/39299
[INFO] processing image 38982/39299
[INFO] processing image 38983/39299
[INFO] processing image 38984/39299
[INFO] processing image 38985/39299
[INFO] processing image 38986/39299
[INFO] processing image 38987/39299
[INFO] processing image 3898

[INFO] processing image 39245/39299
[INFO] processing image 39246/39299
[INFO] processing image 39247/39299
[INFO] processing image 39248/39299
[INFO] processing image 39249/39299
[INFO] processing image 39250/39299
[INFO] processing image 39251/39299
[INFO] processing image 39252/39299
[INFO] processing image 39253/39299
[INFO] processing image 39254/39299
[INFO] processing image 39255/39299
[INFO] processing image 39256/39299
[INFO] processing image 39257/39299
[INFO] processing image 39258/39299
[INFO] processing image 39259/39299
[INFO] processing image 39260/39299
[INFO] processing image 39261/39299
[INFO] processing image 39262/39299
[INFO] processing image 39263/39299
[INFO] processing image 39264/39299
[INFO] processing image 39265/39299
[INFO] processing image 39266/39299
[INFO] processing image 39267/39299
[INFO] processing image 39268/39299
[INFO] processing image 39269/39299
[INFO] processing image 39270/39299
[INFO] processing image 39271/39299
[INFO] processing image 3927

Epoch 5/10
810/810 [==============================] - 17s 21ms/step - loss: nan - acc: 0.9212 - val_loss: nan - val_acc: 0.9215
Epoch 6/10
810/810 [==============================] - 17s 20ms/step - loss: nan - acc: 0.9212 - val_loss: nan - val_acc: 0.9215
Epoch 7/10
810/810 [==============================] - 18s 22ms/step - loss: nan - acc: 0.9212 - val_loss: nan - val_acc: 0.9215
Epoch 8/10
810/810 [==============================] - 18s 22ms/step - loss: nan - acc: 0.9212 - val_loss: nan - val_acc: 0.9215
Epoch 9/10
810/810 [==============================] - 18s 22ms/step - loss: nan - acc: 0.9212 - val_loss: nan - val_acc: 0.9215
Epoch 10/10
810/810 [==============================] - 18s 22ms/step - loss: nan - acc: 0.9212 - val_loss: nan - val_acc: 0.9215
